<a href="https://colab.research.google.com/github/vvithurshan/Homology-Modelling-for-Catnap-Dataset/blob/main/homology_modelling_catnap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import mdtraj as md
from Bio import pairwise2
import MDAnalysis as mda
import numpy as np
from abnumber import Chain
from Bio.Align import substitution_matrices
import pandas as pd
import shutil
from modeller.automodel import *
from modeller import *
import os

In [ ]:
Virus_dict = {}
with open("./CATNAP_DATABASE/Virus_aligned_removed.fasta", 'r') as f:
  for line in f:
    if line.startswith(">"):
      key = line[1:].split('.')[-2]
    else:
      Virus_dict[key] = line[24:510]

In [ ]:
with open('./CATNAP_DATABASE/heavy_seqs_aa.fasta', 'r') as f:
    heavy_dict = {}
    current_id = ""
    for line in f:
        line = line.strip()
        if line.startswith(">"):
            current_id = line[1:].split('_')[0]
            heavy_dict[current_id] = ""
        else:
            heavy_dict[current_id] += line

In [ ]:
with open('./CATNAP_DATABASE/light_seqs_aa.fasta', 'r') as f:
    light_dict = {}
    current_id = ""
    for line in f:
        line = line.strip()
        if line.startswith(">"):
            current_id = line[1:].split('_')[0]
            light_dict[current_id] = ""
        else:
            light_dict[current_id] += line

In [ ]:
def SeqAlignWrite(target_seq, tem_Ab, tem_An):
    tem_Ab_pdb = "{}.pdb".format(tem_Ab)
    tem_An_pdb = "{}.pdb".format(tem_An)
    tem_Ab_pdb_path = "../../../Template_PDB/" + tem_Ab_pdb
    tem_An_pdb_path = "../../../Template_PDB/" + tem_An_pdb

    ## copy template pdb
    shutil.copy(tem_Ab_pdb_path, os.getcwd())
    shutil.copy(tem_An_pdb_path, os.getcwd())

    ##chains to fasta
    tem_ab = mda.Universe(tem_Ab_pdb)
    protein_atoms_ab = tem_ab.select_atoms("protein") 

    template_Ab_chain_ids = [i.segid for i in protein_atoms_ab.segments]

    tem_an = mda.Universe(tem_An_pdb)
    protein_atoms_an = tem_an.select_atoms("protein") 
    template_An_chain_ids = [i.segid for i in protein_atoms_an.segments]

    traj_ab = md.load(tem_Ab_pdb).topology
    template_Ab_chainsToFasta = [traj_ab.to_fasta(i) for i in range(len(template_Ab_chain_ids))]

    traj_an = md.load(tem_An_pdb).topology
    template_An_chainsToFasta = [traj_an.to_fasta(i) for i in range(len(template_An_chain_ids))]

    ## is first a heavy_chain?
    Is_heavy = Chain(template_Ab_chainsToFasta[0] , scheme='chothia', cdr_definition='chothia')

    if not Is_heavy.is_heavy_chain():
        target_seq[0], target_seq[1] = target_seq[1], target_seq[0]
    
    ## row = len of template chain ids
    ## col = len of tar_seq

    row = len(template_Ab_chain_ids) + len(template_An_chain_ids)
    col = len(target_seq)

    score_matrix = np.zeros((row, col))
    ## fill alignment score

    for i, seq_Ab in enumerate(template_Ab_chainsToFasta):
        for j, seq_tar in enumerate(target_seq):
            alignments = pairwise2.align.globalxx(seq_Ab, seq_tar)
            score_matrix[i][j] = alignments[0].score
            # print(alignments[0].score)


    for i, seq_An in enumerate(template_An_chainsToFasta):
        for j, seq_tar in enumerate(target_seq):
            alignments = pairwise2.align.globalxx(seq_An, seq_tar)
            score_matrix[i+len(template_Ab_chainsToFasta)][j] = alignments[0].score
            # print(alignments[0].score)

    ## 
    Target = []
    Template_AB = []
    Template_AN = []
    
    ## alignment parameters
    matrix = substitution_matrices.load("BLOSUM62")
    open_gap_penalty = -10
    extend_gap_penalty = -1
   
    ## do the alignment
    for i in range(col):
        max_val = np.max(score_matrix[:,i])
        max_id = np.where(score_matrix[:,i] == max_val)
        max_id = int(max_id[0])
        if max_id < len(template_Ab_chainsToFasta):
            alignments = pairwise2.align.localds(template_Ab_chainsToFasta[max_id], target_seq[i], matrix, open_gap_penalty, extend_gap_penalty)
            Template_AB.append(alignments[0].seqA)
            Target.append(alignments[0].seqB)
            Template_AN.append(("-" * len(alignments[0].seqA)))
        else:
            alignments = pairwise2.align.localds(template_An_chainsToFasta[max_id - len(template_Ab_chainsToFasta)], target_seq[i], matrix,  open_gap_penalty, extend_gap_penalty)
            Template_AN.append(alignments[0].seqA)
            Target.append(alignments[0].seqB)
            Template_AB.append(("-" * len(alignments[0].seqA)))  

    with open("sequence.ali", "w") as fp:
        ##target
        fp.write(">P1;complex_seq\n")
        fp.write("sequence::.:.:.:.::::\n")
        tar_str = "/".join(Target)
        fp.write(tar_str + "*\n")
        ##ab
        fp.write(f">P1;" + tem_Ab_pdb + "\n")
        fp.write(f"structure:" + tem_Ab_pdb + ":FIRST:@:LAST:.::::\n")
        tem_ab_str = "/".join(Template_AB)
        fp.write(tem_ab_str+"*\n")
        ##an
        fp.write(f">P1;" + tem_An_pdb + "\n")
        fp.write(f"structure:" + tem_An_pdb + ":FIRST:@:LAST:.::::\n")
        tem_an_str = "/".join(Template_AN)
        fp.write(tem_an_str+"*\n")
    
    return (tem_Ab_pdb, tem_An_pdb)

In [ ]:
import os 
import subprocess


## change the directory
def ChangeDir(Antibody, Antigen):

    if not os.path.exists(Antibody):
        os.makedirs(Antibody)
    os.chdir(Antibody)

    folder_name = "{}_{}".format(Antibody,Antigen)
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)
    os.chdir(folder_name)

def Modeller(templates_tuple, output):

    ## modeller 
    env = environ()
    print(templates_tuple)
    a = automodel(env, alnfile='sequence.ali',
                knowns= templates_tuple, sequence='complex_seq')

    a.starting_model = 0
    a.ending_model = 0
    # a.auto_align()

    a.md_level = None
    # a.repeat_optimization = 2
    # a.md_level = refine.slow
    # a.max_var_iterations = 300
    # a.repeat_optimization = 2
    # a.max_molpdf = 1e6
    # a.read_template(template_path)
    a.make()

    model_output = "{}.pdb".format(output)
    shutil.copy("complex_seq.B99990000.pdb", model_output)
    name = 'vvithurshan'
    new_name = 'V _VITHURSHAN'
    subprocess.run(['sed', '-i', f's/{name}/{new_name}/g', model_output])

    storing_location = "/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/CATNAP/Homology_modelling_catnap/PDB_CATNAP"
    shutil.copy(model_output, storing_location)
    return True


def main(Antibody, Antigen, Target_sequence):
    ## change dir
    ChangeDir(Antibody, Antigen)

    ##temple for ab and template for an
    df_tem = pd.read_csv("../../../Ab_An_Template.csv")
    tem_Ab = df_tem[df_tem['Antibody'] == Antibody]['Ab_template'].iloc[0]
    tem_An = df_tem[df_tem['Antibody'] == Antibody]['An_template'].iloc[0]

    templates_tuple = SeqAlignWrite(target_seq = Target_sequence, tem_Ab = tem_Ab, tem_An = tem_An)

    model_out = "{}__{}".format(Antibody, Antigen)

    model = Modeller(templates_tuple = templates_tuple, output = model_out)

    if model:
        return model_out
    else:
        return 0
if __name__ == "__main__":
    ## list of antibodies to be modelled
    os.chdir('/home/vvithurshan/0014/vvarenthirarajah/Documents/ML-Antibody-Antigen-2023/Homology Modeling/CATNAP/Homology_modelling_catnap')
    df = pd.read_csv("../DataBase/test/tst.csv")
    df = df[['Antibody', 'Antigen']]
    list_abs = ['VRC01']
    df_model = df[df['Antibody'].isin(list_abs)]
    # df_model = df[df['Antibody'] == '12A21'] ## temp

    os.chdir('./model/')
    
    for index, row in df_model.iterrows():
        ab = row['Antibody']
        an = row['Antigen']
        tar_seq_list = []
        heavy = heavy_dict[ab].replace("X","")
        light = light_dict[ab].replace("X","")
        tar_seq_list.append(heavy)
        tar_seq_list.append(light)
        try:
            virus_seq = Virus_dict[an].replace("X","")
            tar_seq_list.append(virus_seq)
        except:
            df_model.at[index, 'model'] = 0
            continue
        modeler_model =main(Antibody = ab, Antigen = an, Target_sequence = tar_seq_list)
        df_model.at[index, 'model'] = modeler_model

        os.chdir("../../")
    df_model.to_csv("./VRC01/VRC01.csv")